In [ ]:
# get model
# !git clone https://github.com/multimodallearning/hierarchical-dense-ssl.git

Cloning into 'hierarchical-dense-ssl'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 179 (delta 86), reused 131 (delta 43), pack-reused 0
Receiving objects: 100% (179/179), 57.28 KiB | 4.09 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
# move model to drive for future use
# !mv "/content/hierarchical-dense-ssl" "/content/drive/MyDrive/"

In [ ]:
# get weights
# !wget https://drive.google.com/drive/folders/106mM3_s7m_wu4EoRr-AhilIJ5-r80thU?usp=drive_link

In [ ]:
# get dataset
# !wget https://zenodo.org/records/7155725/files/amos22.zip?download=1 -O amos22.zip

--2024-03-27 13:06:15--  https://zenodo.org/records/7155725/files/amos22.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24234336519 (23G) [application/octet-stream]
Saving to: ‘amos22.zip’

amos22.zip          100%[===================>]  22.57G  24.6MB/s    in 15m 34s 

2024-03-27 13:21:50 (24.7 MB/s) - ‘amos22.zip’ saved [24234336519/24234336519]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
# # __MACOSX isn't needed. Unzip only amos file
# !unzip amos22.zip "amos22/*"

# # rename
# !mv amos22 amos22_extract

'unzip' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# get pipreq for extracting requirements
!pip install pipreqs

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached nbconvert-7.16.3-py3-none-any.whl.metadata (8.2 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached jupyterlab_pygments-0.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-win_amd64.whl.metadata (3.1 kB)
  Using cached mistune-3.0.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached nbclient-0.10.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached pandocfilters-1.5.1-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached tinycss2-1.2.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Us


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# get and download requirements
!pipreqs hierarchical-dense-ssl --force

!pip install -r /hierarchical-dense-ssl/requirements.txt

# other packages that isn't in requirements.txt
!pip install reorient-nii
!pip install clearml
!pip install pytorch_lightning
!pip install amid
!pip install -U tensorboardX
!pip install -U tensorboard

In [ ]:
# dataset.json need to be put in the same dir as amos.zip file, or amid fails to read raw data
# !cp /content/amos22_extract/dataset.json /content/

In [ ]:
# preprocess data for downstream
!python preprocess_downstream_data.py

In [ ]:
!python preprocess_pretrainning_data.py

# changes need in preprocess_downstream_data.py

```
AMOS_DIR = '/content/'
```
```
OUTPUT_DIR = '/content/output/'
```
```
dataset = 'amos'
spacing_x = 1.5  # 1.0 for ct 1.5 for mri
spacing_y = 1.5  # 1.0 for ct 1.5 for mri
spacing_z = 1.5  # 2.0 for ct 1.5 for mri
```






In [ ]:
# !mv /content/output/ /content/drive/MyDrive/

In [ ]:
# !cp -R /content/drive/MyDrive/output/amos/labels /content/drive/MyDrive/output

In [ ]:
# !rm -rf /content/drive/MyDrive/log_dir/eval

In [1]:
# run code
!python run_doownstream_task.py --dataset amos --ckpt /weights/hb_ssl_mri.ckpt --log_dir ./log_dir --batch_size 7 --setup from_scratch

Traceback (most recent call last):
  File "d:\kpdl\code\hierarchical-dense-ssl\run_doownstream_task.py", line 5, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'


# changes need in order to run run_doownstream_task.py

*   in downstream_dataset.py
```
AMOS_DATA_DIR = '/content/output/amos/'
```

# TODO

*   check data split's implementation



